In [5]:
%pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl (198 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.6.15-py3-none-any.whl (157 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
df =pd.read_csv("events.csv")
df

,Event_ID,Event_Name,Event_Date,Outcome,Primary_Type,Secondary_Type,Region,Country_1,Country_2,External_Actors,Key_Individuals,Key_Groups_Entities,Date_Justification,Search_Terms
0,EUR_2022_UKRAINE_CONFLICT,Full-Scale Invasion of Ukraine,2022-02-24,Conflict,Invasion,Interstate War,Europe,Ukraine,Russia,"Belarus,Chinese Hackers,International Criminal...","Putin,Zelenskyy,Biden,Alexander Bortnikov,Niko...","Kremlin,NATO,Russian Army,Ukrainian Army,Donet...",NaN,"Ukraine invasion,Russia special military opera..."
1,MEA_2023_GAZA_CONFLICT,Hamas Attack on Israel,2023-10-07,Conflict,Cross-border Attack,Asymmetric War,Middle East,Israel,Palestine,"Iran,IDF,Gaza,Islamic Jihad","Netanyahu,Sinwar,Mohammed Deif","Hamas,Israel Defense Forces,Al-Qassam Brigades...",NaN,"Hamas attack,Iron Swords,Gaza invasion"
2,AFR_2020_TIGRAY_CONFLICT,Tigray War Start,2020-11-04,Conflict,Civil War,Intervention,Africa,Ethiopia,NaN,"Eritrea,militas from Amhara and Afar","Abiy Ahmed,Debretsion Gebremichael,Getachew Re...","Tigray,TDF,ENDF,Eritrean Defence Forces,TPLF,IRAT",NaN,"Tigray war,Ethiopia conflict,Abiy Ahmed"
3,AFR_2011_LIBYA_CONFLICT,Libyan Civil War Start,2011-02-15,Conflict,Civil War,Foreign Intervention,Africa,Libya,NaN,"NATO,Turkey,UAE,Egypt,France,Wagner Group","Gaddafi,Mustafa Abdul Jalil,Khalifa Haftar,Fay...","National Transitional Council,NTC,UNSC Resolut...",NaN,"Libya civil war,Gaddafi speech,Benghazi uprising"
4,EUR_2008_GEORGIA_CONFLICT,Russo-Georgian War,2008-08-07,Conflict,Invasion,Interstate War,Europe,Georgia,Russia,"USA,NATO,France,Separatist authorities and arm...","Saakashvili,Medvedev,Putin,Borisov,Nicolas Sar...","South Ossetia,Abkhazia,Georgian Armed forces,R...",NaN,"Russo-Georgian war,South Ossetia conflict"
5,MEA_2015_YEMEN_CONFLICT,Saudi-led Intervention in Yemen,2015-03-26,Conflict,Intervention,Proxy War,Middle East,Yemen,Saudi Arabia,"United Arab Emirates, Egypt, Jordan, Morocco, ...","Hadi,King Salman,Mohammed bin Salman,President...","Houthi,Operation Decisive Storm,Saudi-led coal...",NaN,"Saudi intervention Yemen,Houthi conflict"
6,MEA_2011_SYRIA_CONFLICT,Syrian Civil War Begins,2011-03-15,Conflict,Civil War,Proxy War,Middle East,Syria,Saudi Arabia,"Russia,USA,Iran,Turkey,Saudi Arabia,Qatar","Assad,Obama,Abu Mohammed al-Jolani,Putin,Qasse...","Free Syrian Army,FSA,Daraa protests,ISIS,Wagne...",NaN,"Syria civil war,Assad speech,Daraa"
7,EUR_2020_NAGORNO_CONFLICT,Second Nagorno-Karabakh War,2020-09-27,Conflict,Interstate War,NaN,Europe,Armenia,Azerbaijan,"Russia,Turkey,Israel,Syrian mercenaries,Afghan...","Pashinyan,Aliyev,Putin","Nagorno-Karabakh,Artsakh,Armenian Army,Azerbai...",NaN,"Nagorno-Karabakh war,Armenia Azerbaijan conflict"
8,EUR_2014_DONBAS_CONFLICT,Donbas Conflict Start,2014-04-06,Conflict,Hybrid War,Insurgency,Europe,Ukraine,Russia,"OSCE,France,Germany,USA,EU","Turchynov,Strelkov,Viktor Yanukovych,Petro Por...","Donetsk People's Republic,Luhansk People's Rep...",NaN,"Donbas war,Ukraine separatists"
9,EUR_2014_CRIMEA_CONFLICT,Annexation of Crimea,2014-02-27,Conflict,Annexation,Hybrid War,Europe,Ukraine,Russia,"US,EU,NATO,UN,CIS,OSCE","Aksyonov,Putin,Oleksandr Turchynov,Viktor Yanu...","little green men,referendum,Supreme Council of...",NaN,"Crimea annexation,Russia Crimea"


In [34]:
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import dotenv
dotenv.load_dotenv()

# --- CONFIGURATION & CONSTANTS ---
LEAD_UP_DAYS = 90
DATA_DIR = "raw_data"
EVENTS_FILE = "events.csv"

# --- API HELPERS ---

def build_query(event):
    """Builds a robust boolean query for the APIs."""
    
    # 1. Country terms (must have at least one)
    country_terms = f'({event["Country_1"]}'
    if pd.notna(event["Country_2"]):
        country_terms += f' OR {event["Country_2"]}'
    country_terms += ')'

    # 2. Key entities (should have at least one)
    actors = []
    if pd.notna(event["Key_Individuals"]):
        actors.extend([f'{name.strip()}' for name in event["Key_Individuals"].split(',')])
    if  pd.notna(event["Key_Groups_Entities"]):
         actors.extend([f'{name.strip()}' for name in event["Key_Groups_Entities"].split(',')])
    
    actor_terms = " AND (" + " OR ".join(actors) + ")" if actors else ""

    # 3. Thematic terms (from the Search_Terms column)
    theme_terms = ""
    if  pd.notna(event["Search_Terms"]):
        themes = [f'{term.strip()}' for term in event["Search_Terms"].split(',')]
        theme_terms = " AND (" + " OR ".join(themes) + ")"

    # Combine all parts
    full_query = country_terms + actor_terms + theme_terms
    return full_query

def fetch_guardian_data(query, start_date, end_date):
    """Fetches data from The Guardian API, handling pagination."""
    print("  -> Fetching from The Guardian...")
    articles = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        params = {
            'q': query,
            'from-date': start_date,
            'to-date': end_date,
            'api-key': os.getenv('GUARDIAN_API_KEY'),
            'page-size': 50,
            'page': page,
            'show-fields': 'bodyText'
        }
        try:
            response = requests.get('https://content.guardianapis.com/search', params=params)
            response.raise_for_status()
            data = response.json()['response']
            
            for item in data.get('results', []):
                articles.append({
                    'source': 'The Guardian',
                    'date': item['webPublicationDate'],
                    'headline': item['webTitle'],
                    'snippet': item.get('fields', {}).get('bodyText', '')[:500] # Get first 500 chars of body - why?
                })
            
            total_pages = data.get('pages', 1)
            print(f"     Got page {page}/{total_pages}...")
            page += 1
            time.sleep(1) # IMPORTANT: Respect API rate limits
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from The Guardian: {e}")
            break
            
    return articles

def fetch_nyt_data(event, query, start_date_nyt, end_date_nyt):
    """Fetches data from The New York Times API, handling pagination."""
    print("  -> Fetching from The New York Times...")
    articles = []
    page = 0
    
    while True:
        params = {
            'q': query,
            'begin_date': start_date_nyt,
            'end_date': end_date_nyt,
            'api-key': config.NYT_API_KEY,
            'page': page
        }
        try:
            response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params=params)
            response.raise_for_status()
            data = response.json()['response']
            docs = data.get('docs', [])
            
            if not docs:
                break # No more results
            
            for item in docs:
                articles.append({
                    'source': 'The New York Times',
                    'date': item['pub_date'],
                    'headline': item['headline']['main'],
                    'snippet': item.get('abstract', '') or item.get('lead_paragraph', '')
                })
            
            print(f"     Got page {page}...")
            page += 1
            time.sleep(6) # IMPORTANT: NYT has a stricter rate limit (10 reqs/min)
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from NYT: {e}")
            break
            
    return articles


In [ ]:
"""Main function to orchestrate the data acquisition process."""
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

events_df = pd.read_csv(EVENTS_FILE)

for _, event in events_df.iterrows():
    event_id = event['Event_ID']
    output_filename = os.path.join(DATA_DIR, f"{event_id}.csv")

    if os.path.exists(output_filename):
        print(f"Skipping {event_id}: Data file already exists.")
        continue

    print(f"\nProcessing Event: {event_id} - {event['Event_Name']}")

    # Prepare dates and query
    event_date = datetime.strptime(event['Event_Date'], '%Y-%m-%d')
    start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)
    
    start_date_str = start_datetime.strftime('%Y-%m-%d')
    end_date_str = event_date.strftime('%Y-%m-%d')
    
    start_date_nyt = start_datetime.strftime('%Y%m%d')
    end_date_nyt = event_date.strftime('%Y%m%d')
    
    query = build_query(event)
    print(f"  -> Query: {query[:200]}...") # Print first 200 chars of query

    # Fetch data from all sources
    guardian_articles = fetch_guardian_data(event, query, start_date_str, end_date_str)
    nyt_articles = fetch_nyt_data(event, query, start_date_nyt, end_date_nyt)
    
    # Combine and save
    all_articles = guardian_articles + nyt_articles
    if all_articles:
        df = pd.DataFrame(all_articles)
        df.to_csv(output_filename, index=False, encoding='utf-8')
        print(f"Success! Saved {len(df)} articles to {output_filename}")
    else:
        print(f"Warning: No articles found for {event_id}. An empty file will not be created.")
